In [96]:
import pandas as pd
import functools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.random as nr
import math
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm

%matplotlib inline

In [97]:
works_custs = pd.read_csv('AdvWorksCusts.csv')
works_custs.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(works_custs.columns)

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome'],
      dtype='object')


In [98]:
works_custs.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,4/8/66,Bachelors,Professional,M,M,1,0,0,2,137947
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,5/14/65,Bachelors,Professional,M,S,0,1,3,3,101141
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,8/12/65,Bachelors,Professional,M,M,1,1,3,3,91945
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,2/15/68,Bachelors,Professional,F,S,0,1,0,0,86688
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,8/8/68,Bachelors,Professional,F,S,1,4,5,5,92771


In [99]:
avemonth_spend = pd.read_csv('AW_AveMonthSpend.csv')
avemonth_spend.drop_duplicates(subset = 'CustomerID', keep = 'last', inplace = True)
print(avemonth_spend.columns)

Index(['CustomerID', 'AveMonthSpend'], dtype='object')


In [100]:
avemonth_spend.head()

,CustomerID,AveMonthSpend
0,11000,89
1,11001,117
2,11002,123
3,11003,50
4,11004,95


In [101]:
works_custs_avemonth_spend = pd.merge(works_custs, avemonth_spend, how='left', on='CustomerID')
print(works_custs_avemonth_spend.columns)
print(works_custs_avemonth_spend)

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'AveMonthSpend'],
      dtype='object')
       CustomerID Title   FirstName MiddleName    LastName Suffix  \
0           11000   NaN         Jon          V        Yang    NaN   
1           11001   NaN      Eugene          L       Huang    NaN   
2           11002   NaN       Ruben        NaN      Torres    NaN   
3           11003   NaN     Christy        NaN         Zhu    NaN   
4           11004   NaN   Elizabeth        NaN     Johnson    NaN   
5           11005   NaN       Julio        NaN        Ruiz    NaN   
6           11006   NaN       Janet          G     Alvarez    NaN   
7           11007   NaN

In [102]:
print(works_custs_avemonth_spend.shape)
works_custs_avemonth_spend.head()

(16404, 24)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Bachelors,Professional,M,M,1,0,0,2,137947,89
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Bachelors,Professional,M,S,0,1,3,3,101141,117
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Bachelors,Professional,M,M,1,1,3,3,91945,123
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Bachelors,Professional,F,S,0,1,0,0,86688,50
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Bachelors,Professional,F,S,1,4,5,5,92771,95


In [103]:
labels = np.array(works_custs_avemonth_spend['AveMonthSpend'])
labels.size

16404

In [104]:
def encode_string(cat_features):
    ## First encode the strings to numeric categories
    enc = preprocessing.LabelEncoder()
    enc.fit(cat_features)
    enc_cat_features = enc.transform(cat_features)
    ## Now, apply one hot encoding
    ohe = preprocessing.OneHotEncoder()
    encoded = ohe.fit(enc_cat_features.reshape(-1,1))
    return encoded.transform(enc_cat_features.reshape(-1,1)).toarray()

categorical_columns = ['Education', 'Gender', 'MaritalStatus']

Features = encode_string(works_custs_avemonth_spend['Occupation'])
for col in categorical_columns:
    temp = encode_string(works_custs_avemonth_spend[col])
    Features = np.concatenate([Features, temp], axis = 1)

print(Features.shape)
print(Features[:2, :])    

(16404, 14)
[[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1.]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [105]:
Features = np.concatenate([Features, np.array(works_custs_avemonth_spend[['HomeOwnerFlag','NumberCarsOwned', 
                            'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome']])], axis = 1)
print(Features.shape)
print(Features[:2, :])

(16404, 19)
[[0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00
  1.000e+00 0.000e+00 0.000e+00 2.000e+00 1.379e+05]
 [0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 1.000e+00
  0.000e+00 1.000e+00 3.000e+00 3.000e+00 1.011e+05]]


In [106]:
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 1000) #need to have shuffle=False in here for correct answer
X_train = Features[indx[0],:]
y_train = np.ravel(labels[indx[0]])
#X_test = Features[indx[1],:]
#y_test = np.ravel(labels[indx[1]])

In [107]:
y_train

array([ 70,  74,  78, ...,  73, 127,  46])

In [108]:
from sklearn.feature_selection import SelectKBest

from sklearn.feature_selection import chi2

test = SelectKBest(score_func=chi2, k=4)

In [109]:
fit = test.fit(X_train, y_train)

In [110]:
features = fit.transform(X_train)
np.set_printoptions(precision=3)
print(fit.scores_)

[7.386e+02 7.859e+02 1.715e+03 1.053e+03 3.856e+02 4.133e+02 4.607e+02
 3.783e+02 2.535e+02 5.511e+02 4.182e+03 4.085e+03 6.005e+02 7.101e+02
 2.225e+02 2.108e+03 2.142e+04 6.198e+03 1.346e+08]


In [111]:
X_test1 = Features[indx[1],:]
y_test1 = np.ravel(labels[indx[1]])

In [112]:
scaler = preprocessing.StandardScaler().fit(X_train[:,18:])
X_train[:,18:] = scaler.transform(X_train[:,18:])
X_test1[:,18:] = scaler.transform(X_test1[:,18:])
X_train[:2,]

array([[ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         1.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,  1.   ,  1.   ,
         0.   ,  1.   , -0.715],
       [ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         1.   ,  0.   ,  1.   ,  0.   ,  0.   ,  1.   ,  1.   ,  3.   ,
         4.   ,  4.   , -0.631]])

In [113]:
lin_mod = linear_model.LinearRegression(fit_intercept = False)
lin_mod.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [114]:
y_score = lin_mod.predict(X_test1) 
print(np.array(y_score[:500]))

[ 85.248  54.264  46.356  97.95   82.465  44.574  76.365  73.899  30.524
  47.331  41.027  70.259  63.511  48.166  44.959  78.037  65.61   49.817
  95.613  60.49   65.453  49.778  59.846  90.021 122.288  52.146 135.539
 126.705  50.453  38.984  56.144  75.17   62.117  81.514  75.187  60.956
  42.91  142.834  47.549 103.815  34.994  84.943  44.904  92.888  68.457
  62.435  73.866  64.276  50.942 117.804  87.017  81.176  35.972 107.848
  68.99   49.457  77.5    54.403  77.266  65.404  72.694  40.491  69.52
 112.359 150.329  68.503  80.784  65.59   35.764  43.335 107.982  80.528
  51.512  54.625  70.256  36.674  40.703  47.436  56.758  54.667  74.377
  79.561  88.173  40.672  77.668  66.11   54.039  32.523  43.406  67.923
  69.363  49.342  43.099  30.062  64.904 135.646  37.569  37.881  63.523
  87.502  76.452  47.035  74.138  63.81   43.789  87.987  39.997 105.88
  38.914  72.544  66.744  72.14   82.072 144.84  107.508  63.246  55.783
  49.332  36.078  79.791  72.492  83.922  84.356  42.

In [115]:
aw_test = pd.read_csv('aw_test.csv')
print(aw_test.shape)
aw_test.head()

(500, 23)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,18988,NaN,Courtney,A,Baker,NaN,8727 Buena Vista Ave.,NaN,Fremont,California,...,1/5/1945,Bachelors,Management,F,S,0,2,0,5,86931
1,29135,NaN,Adam,C,Allen,NaN,3491 Cook Street,NaN,Haney,British Columbia,...,10/4/1964,Bachelors,Skilled Manual,M,M,1,2,2,4,100125
2,12156,NaN,Bonnie,NaN,Raji,NaN,359 Pleasant Hill Rd,NaN,Burbank,California,...,1/12/1934,Graduate Degree,Management,F,M,1,2,0,4,103985
3,13749,NaN,Julio,C,Alonso,NaN,8945 Euclid Ave.,NaN,Burlingame,California,...,9/22/1958,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161
4,27780,NaN,Christy,A,Andersen,NaN,"42, boulevard Tremblay",NaN,Dunkerque,Nord,...,3/19/1965,High School,Manual,F,M,1,1,2,2,21876


In [116]:
categorical_columns

['Education', 'Gender', 'MaritalStatus']

In [117]:
temp

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [118]:
FeaturesT = encode_string(aw_test['Occupation'])
for col in categorical_columns:
    tempT = encode_string(aw_test[col])
    FeaturesT = np.concatenate([FeaturesT, tempT], axis = 1)

print(FeaturesT.shape)
print(FeaturesT[:20, :])   

(500, 14)
[[0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1.]]


/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [119]:
FeaturesT = np.concatenate([FeaturesT, np.array(aw_test[['HomeOwnerFlag','NumberCarsOwned', 
                            'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome']])], axis = 1)
print(FeaturesT.shape)
print(FeaturesT[:2, :]) 

(500, 19)
[[0.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00
  0.000e+00 2.000e+00 0.000e+00 5.000e+00 8.693e+04]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 1.000e+00 0.000e+00
  1.000e+00 2.000e+00 2.000e+00 4.000e+00 1.001e+05]]


In [120]:
indxT = range(FeaturesT.shape[0])
indxT = ms.train_test_split(indxT, test_size=0, shuffle=False)
X_test = FeaturesT[indxT[0],:]

#y_test = np.ravel(labelsT[indxT[0]])
X_test[:19, :19]

array([[0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00, 2.000e+00, 0.000e+00, 5.000e+00,
        8.693e+04],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 2.000e+00, 2.000e+00, 4.000e+00,
        1.001e+05],
       [0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 2.000e+00, 0.000e+00, 4.000e+00,
        1.040e+05],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        1.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 4.000e+00,
        1.272e+05],
       [0.000e+00, 0.000e+00, 1.

In [121]:
scaler = preprocessing.StandardScaler().fit(X_test[:,18:])
X_test[:,18:] = scaler.transform(X_test[:,18:])
X_test[:19,:19]

array([[ 0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  1.   ,  0.   ,  2.   ,
         0.   ,  5.   ,  0.127],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,  1.   ,  2.   ,
         2.   ,  4.   ,  0.452],
       [ 0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  1.   ,  0.   ,  1.   ,  0.   ,  1.   ,  2.   ,
         0.   ,  4.   ,  0.547],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  1.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  1.   ,  1.   ,  0.   ,  1.   ,  0.   ,
         0.   ,  4.   ,  1.118],
       [ 0.   ,  0.   ,  1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.   ,
         0.   ,  0.   ,  1.   ,  0.   ,  1.   ,  0.   ,  1.   ,  1.   ,
         2.   ,  2.   , -1.476],
       [ 1.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         1.   ,  0.   ,  1.   ,  0.   ,  1.

In [122]:
X_test.shape

(500, 19)

In [123]:
X_train

array([[ 1.   ,  0.   ,  0.   , ...,  0.   ,  1.   , -0.715],
       [ 1.   ,  0.   ,  0.   , ...,  4.   ,  4.   , -0.631],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  5.   ,  0.762],
       ...,
       [ 1.   ,  0.   ,  0.   , ...,  0.   ,  4.   , -0.096],
       [ 0.   ,  1.   ,  0.   , ...,  4.   ,  4.   ,  1.132],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   , -0.743]])

In [124]:
X_test

array([[ 0.   ,  1.   ,  0.   , ...,  0.   ,  5.   ,  0.127],
       [ 0.   ,  0.   ,  0.   , ...,  2.   ,  4.   ,  0.452],
       [ 0.   ,  1.   ,  0.   , ...,  0.   ,  4.   ,  0.547],
       ...,
       [ 0.   ,  1.   ,  0.   , ...,  3.   ,  3.   ,  1.387],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  2.   ,  0.485],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  4.   , -1.545]])

In [125]:
lin_mod = linear_model.LinearRegression(fit_intercept = False)
lin_mod.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

In [126]:
X_test.shape

(500, 19)

In [127]:
y_score = lin_mod.predict(X_test) 
print(np.array(y_score[:500]))

[ 41.349 105.056  49.22   87.178  60.246  42.889  94.34  126.525 102.612
  55.372  59.201  48.955  72.425  45.17   36.117  50.92   84.268  73.23
 109.142  58.542  67.8    75.764 147.353  85.083  51.236  75.4    86.084
 115.616  77.819  59.604  69.02   79.497  40.175  70.429 105.548 102.671
 146.128  91.781  57.465  86.58   46.496  80.424  80.835  47.848  57.242
  75.322  61.236  87.471 115.101  78.138  81.486  94.87   79.07   64.967
  46.416  75.878  58.516  74.52   64.423  68.25   44.441  64.711  91.391
  81.886  43.676  81.165  81.298 131.006  63.937 106.761  84.853  69.792
  93.999  43.038  66.999  81.795 113.114  69.075  59.997  45.726  87.393
  73.26   45.702  88.868  85.344  48.247  82.658 119.871  97.357  53.36
  53.165  94.5    36.894  48.049  37.489 139.077  58.78   48.459  45.17
  52.833 103.727  74.842  49.217 118.843  83.777 104.121  32.333  45.65
  46.028  59.349  94.688  46.731  84.696  63.008  75.323  86.197  37.32
  39.296  46.353  88.068 123.782  87.082  81.22   38.578

In [128]:
df = pd.DataFrame(y_score[:500])
df.head()

,0
0,41.349121
1,105.056152
2,49.219727
3,87.177734
4,60.246094


In [129]:
df.to_csv('aw_test_prediction.csv')